In [4]:
import copy 
import json

ATTACK = '1'
BENIGN = '0'
OTHERS = '-1'

with open("../../data/pre_process/dissection_clean/5GCTD2.json") as f:
    my_json = json.load(f)

dict_by_type = {
    "discovery" : [],
    "manageNF"  : [],
    "apiCall"   : [],
    "sessionDeletion"      : [],
    "sessionModification"  : [],
    "sessionEstablishment" : [],
    "gtpMessage"         : [],
}

dict = {
    ATTACK: copy.deepcopy(dict_by_type),
    BENIGN: copy.deepcopy(dict_by_type),
    OTHERS: copy.deepcopy(dict_by_type)
}

for dissected_pkt in my_json:

    is_attack = dissected_pkt["common"]["is_attack"]

    if "http2" in dissected_pkt["protocols"]:

        for layer in dissected_pkt["protocols"]["http2"]:

            if "path" in layer:

                # Discovery Requests
                if "/nnrf-disc" in layer["path"]:
                    dict[is_attack]["discovery"].append(dissected_pkt)

                # Add and remove NF Requests
                if "/nnrf-nfm" in layer["path"]:
                    dict[is_attack]["manageNF"].append(dissected_pkt)

                dict[is_attack]["apiCall"].append(dissected_pkt)

    if "pfcp" in dissected_pkt["protocols"]:

        for layer in dissected_pkt["protocols"]["pfcp"]:

            if "msg_type" in layer:

                # PFCP Deletion Requests
                if int(layer["msg_type"]) == 54:
                    dict[is_attack]["sessionDeletion"].append(dissected_pkt)

                # PFCP Modification Requests
                if int(layer["msg_type"]) == 52:
                    dict[is_attack]["sessionModification"].append(dissected_pkt)

                # PFCP Establishment Requests
                if int(layer["msg_type"]) == 50:
                    dict[is_attack]["sessionEstablishment"].append(dissected_pkt)

    if "gtp" in dissected_pkt["protocols"]:
        dict[is_attack]["gtpMessage"].append(dissected_pkt)

print("BENIGN MESSAGES PER ATTACK TYPE\n\n")
print("ATTACKS")
for key,value in dict[ATTACK].items():
    print(key, len(value))

print("-------")

print("BENIGN")
for key,value in dict[BENIGN].items():
    print(key, len(value))

print("-------")

print("OTHERS")
for key,value in dict[OTHERS].items():
    print(key, len(value))

BENIGN MESSAGES PER ATTACK TYPE


ATTACKS
discovery 0
manageNF 0
apiCall 0
sessionDeletion 4122
sessionModification 3959
sessionEstablishment 1900
gtpMessage 47
-------
BENIGN
discovery 0
manageNF 0
apiCall 0
sessionDeletion 0
sessionModification 0
sessionEstablishment 0
gtpMessage 172
-------
OTHERS
discovery 0
manageNF 0
apiCall 0
sessionDeletion 0
sessionModification 0
sessionEstablishment 99
gtpMessage 0
